In [38]:
import sys
from pathlib import Path

project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))


In [39]:
from src.station import Station
from src.network import ChargingNetwork
from src.io_utils import load_acn_csv
from src.analysis import compute_utilization_by_station

print("Imports successful!")


Imports successful!


In [40]:
import json
import pandas as pd
from pathlib import Path

json_path = Path("../data/raw/acn_sessions.json")

# Load JSON
with open(json_path, "r") as f:
    data = json.load(f)

# Extract sessions from ACN structure
sessions = data["_items"]

# Convert to DataFrame
df = pd.DataFrame(sessions)

# Save as CSV
csv_path = Path("../data/raw/acn_sessions.csv")
df.to_csv(csv_path, index=False)

print("✅ JSON converted to CSV successfully")
print("Rows:", len(df))
print("Columns:", df.columns.tolist())


✅ JSON converted to CSV successfully
Rows: 10607
Columns: ['_id', 'clusterID', 'connectionTime', 'disconnectTime', 'doneChargingTime', 'kWhDelivered', 'sessionID', 'siteID', 'spaceID', 'stationID', 'timezone', 'userID', 'userInputs']


In [41]:
import pandas as pd
from pathlib import Path

# Load processed CSV
csv_path = Path("../data/raw/acn_sessions.csv")

df = pd.read_csv(csv_path)

print("Dataset loaded successfully")
print("Shape:", df.shape)
df.head()


Dataset loaded successfully
Shape: (10607, 13)


,_id,clusterID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,5c427db1f9af8b1e1a538287,39,"Wed, 02 Jan 2019 13:53:10 GMT","Wed, 02 Jan 2019 22:38:52 GMT","Wed, 02 Jan 2019 19:50:11 GMT",36.608,2_39_127_19_2019-01-02 13:53:10.182304,2,CA-309,2-39-127-19,America/Los_Angeles,556.0,"[{'WhPerMile': 300, 'kWhRequested': 63.0, 'mil..."
1,5c427db1f9af8b1e1a538288,39,"Wed, 02 Jan 2019 15:53:25 GMT","Thu, 03 Jan 2019 00:38:37 GMT","Wed, 02 Jan 2019 19:05:25 GMT",8.855,2_39_79_382_2019-01-02 15:53:25.247434,2,CA-491,2-39-79-382,America/Los_Angeles,1001.0,"[{'WhPerMile': 240, 'kWhRequested': 12.0, 'mil..."
2,5c427db1f9af8b1e1a538289,39,"Wed, 02 Jan 2019 16:12:39 GMT","Thu, 03 Jan 2019 01:06:11 GMT","Wed, 02 Jan 2019 19:10:32 GMT",10.402,2_39_78_361_2019-01-02 16:12:38.806280,2,CA-493,2-39-78-361,America/Los_Angeles,1110.0,"[{'WhPerMile': 400, 'kWhRequested': 36.0, 'mil..."
3,5c427db1f9af8b1e1a53828a,39,"Wed, 02 Jan 2019 16:18:36 GMT","Wed, 02 Jan 2019 23:53:56 GMT","Wed, 02 Jan 2019 23:53:52 GMT",46.058,2_39_91_441_2019-01-02 16:18:35.672653,2,CA-499,2-39-91-441,America/Los_Angeles,234.0,"[{'WhPerMile': 250, 'kWhRequested': 62.5, 'mil..."
4,5c427db1f9af8b1e1a53828b,39,"Wed, 02 Jan 2019 16:18:56 GMT","Thu, 03 Jan 2019 02:17:39 GMT","Thu, 03 Jan 2019 02:17:34 GMT",39.007,2_39_79_379_2019-01-02 16:18:55.665332,2,CA-327,2-39-79-379,America/Los_Angeles,232.0,"[{'WhPerMile': 391, 'kWhRequested': 50.83, 'mi..."


In [42]:
# Convert time columns to datetime
time_cols = ["connectionTime", "disconnectTime"]

for col in time_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")

print(df[time_cols].dtypes)


connectionTime    datetime64[ns]
disconnectTime    datetime64[ns]
dtype: object


In [43]:
from src.network import ChargingNetwork
import pandas as pd

network = ChargingNetwork(name="Caltech ACN")

for _, row in df.iterrows():
    if pd.notna(row["connectionTime"]) and pd.notna(row["disconnectTime"]):
        network.add_session(
            station_id=row["stationID"],
            start=row["connectionTime"],
            end=row["disconnectTime"]
        )

print(f"Stations loaded: {len(network.stations)}")


Stations loaded: 54


In [44]:
from datetime import datetime

window_start = datetime(2019, 1, 1)
window_end = datetime(2019, 12, 31, 23, 59, 59)

utilization_results = []

for station_id, station in network.stations.items():
    util = station.compute_utilization(window_start, window_end)
    utilization_results.append({
        "stationID": station_id,
        "utilization": util
    })

import pandas as pd
util_df = pd.DataFrame(utilization_results).sort_values(
    by="utilization", ascending=False
)

util_df.head()


,stationID,utilization
11,2-39-139-28,0.402559
10,2-39-131-30,0.358498
14,2-39-89-25,0.314344
12,2-39-91-437,0.299272
4,2-39-79-379,0.278258


In [45]:
overused = util_df[util_df["utilization"] >= 0.35]
underused = util_df[util_df["utilization"] <= 0.15]

print("Overused stations:")
display(overused)

print("Underused stations:")
display(underused)


Overused stations:


,stationID,utilization
11,2-39-139-28,0.402559
10,2-39-131-30,0.358498


Underused stations:


,stationID,utilization
36,2-39-124-22,0.148564
40,2-39-78-360,0.138411
22,2-39-78-367,0.131842
25,2-39-78-364,0.128339
44,2-39-78-362,0.126020
24,2-39-90-440,0.124696
29,2-39-130-31,0.123988
2,2-39-78-361,0.118480
18,2-39-95-444,0.118326
3,2-39-91-441,0.117891


In [46]:
from datetime import datetime

recs = network.recommend_actions(
    window_start=datetime(2019, 1, 1),
    window_end=datetime(2019, 12, 31, 23, 59, 59)
)

for r in recs[:5]:
    print("-", r)


- Station 2-39-78-361 is underused (0.12). Consider maintenance or incentives.
- Station 2-39-91-441 is underused (0.12). Consider maintenance or incentives.
- Station 2-39-139-567 is underused (0.00). Consider maintenance or incentives.
- Station 2-39-130-564 is underused (0.01). Consider maintenance or incentives.
- Station 2-39-95-444 is underused (0.12). Consider maintenance or incentives.


In [47]:
while True:
    print("\nEV Charging Station Utilization Analyzer")
    print("1. Show top utilized stations")
    print("2. Show recommendations")
    print("3. Exit")

    choice = input("Enter choice: ")

    if choice == "1":
        display(util_df.head())
    elif choice == "2":
        for r in recs:
            print("-", r)
    elif choice == "3":
        print("Exiting program.")
        break
    else:
        print("Invalid choice.")



EV Charging Station Utilization Analyzer
1. Show top utilized stations
2. Show recommendations
3. Exit


Enter choice:  1


,stationID,utilization
11,2-39-139-28,0.402559
10,2-39-131-30,0.358498
14,2-39-89-25,0.314344
12,2-39-91-437,0.299272
4,2-39-79-379,0.278258



EV Charging Station Utilization Analyzer
1. Show top utilized stations
2. Show recommendations
3. Exit


Enter choice:  3


Exiting program.
